# CGV 무비차트 크롤링

In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
url = "http://www.cgv.co.kr/movies/?lt=1&ft=0"
header = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Trident/7.0; rv:11.0) like Gecko'}
req = requests.get(url, headers=header)
soup = BeautifulSoup(req.text, 'html.parser')

## 1. 필요한 데이터 찾기

In [14]:
# 무비 차트 태그를 찾아서 리스트로 저장하기
movie_list = soup.select('.sect-movie-chart')[0].find_all('li')

In [15]:
# 영화 한개 고르기
movie = movie_list[2]

In [16]:
# 제목
title = movie.select_one('.box-contents').find('strong').text.strip()
title

'교섭'

In [18]:
# 예매율
try:
    percent = float(movie.select_one('.score').find('span').text.replace('%', ''))
except:
    percent = 0.0

In [19]:
# 평점
try:
    rating = float(movie.select_one('.egg-gage > span.percent').text.replace('%', ''))
except:
    rating = float('NaN')

In [20]:
# 개봉일
release_date = movie.select_one('.txt-info').text.strip().split(" ")[0]
release_date

'2023.01.18'

## 2. 리스트에 저장

In [10]:
chart = []
for i, movie in enumerate(soup.select('.sect-movie-chart')[0].find_all('li')):
    title = movie.select_one('.box-contents').find('strong').text.strip() # 제목
    try:
        percent = float(movie.select_one('.score').find('span').text.replace('%', '')) # 예매율
    except:
        percent = 0.0
    try:
        rating = float(movie.select_one('.egg-gage > span.percent').text.replace('%', '')) # 평점
    except:
        rating = float('NaN')
    release_date = movie.select_one('.txt-info').text.strip().split(" ")[0] # 개봉일
    chart.append({
        '순위' : i+1, '제목' : title, '개봉일' : release_date, '예매율' : percent, '평점' : rating
    })

## 3. 데이터 프레임으로 변환

In [11]:
df_movie = pd.DataFrame(chart)
df_movie

,순위,제목,개봉일,예매율,평점
0,1,아바타-물의 길,2022.12.14,47.0,96.0
1,2,방탄소년단: 옛 투 컴 인 시네마,2023.02.01,11.7,99.0
2,3,교섭,2023.01.18,8.9,99.0
3,4,더 퍼스트 슬램덩크,2023.01.04,8.3,97.0
4,5,유령,2023.01.18,7.3,99.0
5,6,영웅,2022.12.21,4.1,94.0
6,7,장화신은 고양이-끝내주는 모험,2023.01.04,2.7,96.0
7,8,"오늘 밤, 세계에서 이 사랑이 사라진다 해도",2022.11.30,1.6,91.0
8,9,스위치,2023.01.04,1.6,94.0
9,10,웃는남자 감독판,2023.01.02,1.4,97.0


## 4. csv 파일로 변환

In [12]:
df_movie.to_csv('CGV무비차트.csv')